## Data cleaning

In [135]:
# library imports

import numpy as np
import pandas as pd
import csv
import datetime as dt
import statistics
import time
from ast import literal_eval
import itertools
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## SteamSpy data

In [4]:
raw_steamspy_data = pd.read_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steamspy_data_raw.csv')
raw_steamspy_data.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,60,Ricochet,Valve,Valve,NaN,4218,929,0,"5,000,000 .. 10,000,000",24,0,3,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,3,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110..."
1,20,Team Fortress Classic,Valve,Valve,NaN,6357,979,0,"5,000,000 .. 10,000,000",136,0,20,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,75,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271..."
2,10,Counter-Strike,Valve,Valve,NaN,214651,5470,0,"10,000,000 .. 20,000,000",7720,134,232,147,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,10895,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3..."
3,70,Half-Life,Valve,Valve,NaN,85453,3070,0,"2,000,000 .. 5,000,000",391,109,168,184,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,525,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ..."
4,80,Counter-Strike: Condition Zero,Valve,Valve,NaN,20911,1995,0,"5,000,000 .. 10,000,000",617,1,71,1,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,232,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ..."


In [21]:
raw_steamspy_data.shape

(61164, 20)

In [22]:
raw_steamspy_data.dtypes

appid                int64
name                object
developer           object
publisher           object
score_rank         float64
positive             int64
negative             int64
userscore            int64
owners              object
average_forever      int64
average_2weeks       int64
median_forever       int64
median_2weeks        int64
price              float64
initialprice       float64
discount           float64
languages           object
genre               object
ccu                  int64
tags                object
dtype: object

In [5]:
#checking Null values in all columns
raw_steamspy_data.isnull().sum()

appid                  0
name                   9
developer            229
publisher            152
score_rank         61121
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             84
genre                289
ccu                    0
tags                   0
dtype: int64

In [7]:
#name column
raw_steamspy_data[raw_steamspy_data['name'].isnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
476,17760,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
10647,506630,NaN,NaN,Big Fish Games,NaN,1,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Action': 21, 'Adventure': 21}"
13236,576960,NaN,Nuclear Tales,NaN,NaN,110,26,0,"20,000 .. 50,000",0,0,0,0,299.0,1499.0,80.0,"English, Spanish - Spain, French, German","Adventure, Casual, Indie, RPG, Simulation, Str...",0,"{'Adventure': 377, 'Choose Your Own Adventure'..."
21544,806220,NaN,"Jochen Heizmann, Emmanuel Henné, Intermediaware",Asylum Square,NaN,26,13,0,"20,000 .. 50,000",0,0,0,0,0.0,0.0,0.0,English,"Action, Indie",0,"{'Side Scroller': 102, ""Shoot 'Em Up"": 97, 'Bu..."
27188,965340,NaN,2nd Studio,2nd Studio,NaN,55,7,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,"English, German, Danish, Japanese, Russian, Si...","Action, Indie, Simulation",0,"{'Indie': 34, 'Action': 31, 'Sexual Content': ..."
30890,1071920,NaN,Ultimo Games,Ultimo Games,NaN,19,0,0,"0 .. 20,000",0,0,0,0,399.0,399.0,0.0,English,"Action, Indie",0,"{'Indie': 33, 'Action': 33, 'Retro': 16, 'Arca..."
32396,1116910,NaN,NaN,NaN,NaN,12,16,0,"0 .. 20,000",0,0,0,0,699.0,699.0,0.0,NaN,"Action, Adventure, Casual, Indie, RPG, Simulat...",0,"{'Casual': 33, 'Adventure': 31, 'Indie': 30, '..."
35742,1216770,NaN,NaN,Driven Arts,NaN,1,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, French",NaN,0,"{'Casual': 104, 'Indie': 98, 'RPGMaker': 94, '..."
52840,1860160,NaN,NaN,Mens Sana Interactive,NaN,2,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,English,NaN,0,"{'FPS': 286, 'PvE': 279, 'Shooter': 275, 'Acti..."


In [9]:
#name column
raw_steamspy_data[raw_steamspy_data['name'] == 'none']

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4839,339860,none,NaN,Green Sauce Games,NaN,84,37,0,"50,000 .. 100,000",322,0,324,0,0.0,0.0,0.0,English,NaN,0,"{'Indie': 53, 'RPG': 33, 'Adventure': 32, 'Mas..."
6620,385020,none,none,Valsar,NaN,13,33,0,"20,000 .. 50,000",261,0,263,0,0.0,0.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie, Massively Multiplayer, Strategy",1,"{'Indie': 47, 'Strategy': 31, 'Casual': 21, 'M..."
7140,398970,none,none,none,NaN,8,23,0,"50,000 .. 100,000",255,0,261,0,0.0,0.0,0.0,English,"Adventure, Indie, Massively Multiplayer, RPG, ...",0,"{'Massively Multiplayer': 38, 'Strategy': 23, ..."


In [12]:
#developer
raw_steamspy_data[raw_steamspy_data['developer'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
14052,600600,MakeThatMoney,NaN,My Way Games,NaN,21,25,0,"0 .. 20,000",242,0,242,0,99.0,99.0,0.0,English,NaN,0,"{'Simulation': 67, 'Casual': 61, 'Top-Down': 4..."
31851,1100510,Demo,NaN,nightowls,NaN,101,41,0,"20,000 .. 50,000",0,0,0,0,0.0,0.0,0.0,English,NaN,0,"{'Free to Play': 37, 'Action': 34, 'Violent': ..."
8500,439660,Tower Unite Dedicated Server,NaN,MLB.com,NaN,3,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Free to Play': 40, 'Mini Golf': 8}"
25328,910430,The War of the Worlds: Andromeda,NaN,Mediamas Europa,NaN,5,11,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,"English, Spanish - Spain","Action, Indie",0,"{'Action': 21, 'Indie': 21}"
14556,615490,I'm Awesome,NaN,Team D-13,NaN,13,8,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,English,NaN,0,"{'Action': 35, 'Indie': 33, 'Casual': 33, 'Vio..."


In [15]:
#publisher
raw_steamspy_data[raw_steamspy_data['publisher'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
16995,684740,Dungeons 3 Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
2107,239490,America's Army: Proving Grounds Beta (Closed),NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,0.0,0.0,NaN,NaN,1,[]
6146,371510,Gunnheim,SIEIDI Ltd,NaN,NaN,128,191,0,"50,000 .. 100,000",161,0,243,0,999.0,999.0,0.0,English,"Action, Indie",0,"{'Action': 69, 'Indie': 67, 'Co-op': 42, 'Twin..."
1228,72530,Arcadia Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
17933,709970,SwapQuest Demo,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]


In [13]:
#price
raw_steamspy_data[raw_steamspy_data['price'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4221,321210,Blade Symphony Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
17582,700580,Rust - Staging Branch,NaN,NaN,NaN,0,0,0,"0 .. 20,000",266,0,3,0,NaN,NaN,NaN,NaN,NaN,5,[]
21811,813350,Ben 10 VR,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
15046,630790,PSYCHO-PASS,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,1,[]
11580,530940,BIOHAZARD 7 resident evil グロテスクVer.,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,20,[]


In [17]:
raw_steamspy_data[raw_steamspy_data['languages'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
1398,200350,Starvoid,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
1705,215590,Endless Space QA,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
1877,226760,Rugby Challenge 2,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
2438,254270,Dungeonland - All access pass,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
1221,71310,Football Manager 2012 Korean,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,20,[]


In [14]:
#genre
raw_steamspy_data[raw_steamspy_data['genre'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
1431,201930,Jamestown IGF,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
15975,661250,Jarheads,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
2145,242360,Football Manager 2014 Korean,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,18,[]
20671,783140,Removed,NaN,Slitherine Ltd.,NaN,25,16,0,"200,000 .. 500,000",36,0,22,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Simulation': 1105, 'Action': 35, 'Indie': 33}"
35698,1215390,Animal Revolt Battle Simulator,VDimension,"VDimension, Yodo1 Ltd.",NaN,4151,167,0,"200,000 .. 500,000",499,0,547,0,1499.0,1499.0,0.0,"English, French, Italian, German, Spanish - Sp...",NaN,242,"{'Indie': 384, 'Sandbox': 365, 'Dinosaurs': 33..."


In [23]:
#checking number of actual values in column
raw_steamspy_data['positive'].value_counts().head()

1    4076
2    3265
3    2709
4    2480
0    2224
Name: positive, dtype: int64

In [24]:
raw_steamspy_data['negative'].value_counts().head()

0    10018
1     7241
2     4823
3     3422
4     2623
Name: negative, dtype: int64

In [25]:
raw_steamspy_data['userscore'].value_counts().head()

0      61121
100        4
95         3
80         2
94         2
Name: userscore, dtype: int64

In [27]:
raw_steamspy_data['average_forever'].value_counts().head()

0    47826
1      315
2      112
4      102
3       99
Name: average_forever, dtype: int64

In [28]:
raw_steamspy_data['average_2weeks'].value_counts().head()

0    59741
1       84
2       26
4       23
3       19
Name: average_2weeks, dtype: int64

In [29]:
raw_steamspy_data['median_forever'].value_counts().head()

0    47826
1      314
2      103
4      100
3       99
Name: median_forever, dtype: int64

In [30]:
raw_steamspy_data['median_2weeks'].value_counts().head()

0    59741
1       83
2       25
4       24
3       19
Name: median_2weeks, dtype: int64

In [31]:
raw_steamspy_data['ccu'].value_counts().head()

0    44496
1     5579
2     2002
3     1197
4      798
Name: ccu, dtype: int64

### Removing columns

`score_rank` has too many missing values, `userscore` most values are 0, `average_2weeks`, `median_2weeks` (average and median playtime in the last 2 weeks) and `ccu`(peak conccurrent users yesterday) are time specific and there's no historic data for them in dataset

In [35]:
steamspy_data = raw_steamspy_data.drop(columns=['score_rank', 'userscore', 'average_2weeks', 'median_2weeks', 'ccu'])

In [36]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price,initialprice,discount,languages,genre,tags
0,60,Ricochet,Valve,Valve,4218,929,"5,000,000 .. 10,000,000",24,3,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110..."
1,20,Team Fortress Classic,Valve,Valve,6357,979,"5,000,000 .. 10,000,000",136,20,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271..."
2,10,Counter-Strike,Valve,Valve,214651,5470,"10,000,000 .. 20,000,000",7720,232,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3..."
3,70,Half-Life,Valve,Valve,85453,3070,"2,000,000 .. 5,000,000",391,168,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ..."
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,"5,000,000 .. 10,000,000",617,71,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ..."


### Cleaning columns and rows

#### Owners column

In [41]:
steamspy_data['owners'].head()

0     5,000,000 .. 10,000,000
1     5,000,000 .. 10,000,000
2    10,000,000 .. 20,000,000
3      2,000,000 .. 5,000,000
4     5,000,000 .. 10,000,000
Name: owners, dtype: object

In [42]:
# Splitting the owners column into lower and upper bound of an estimation for the amount of owners for the game
steamspy_data[['owners_lowb', 'owners_uppb']] = steamspy_data['owners'].str.replace(',', '').str.split(' .. ', expand=True)

In [44]:
#drop original owners column
steamspy_data = steamspy_data.drop(columns=['owners'])

In [45]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,average_forever,median_forever,price,initialprice,discount,languages,genre,tags,owners_lowb,owners_uppb
0,60,Ricochet,Valve,Valve,4218,929,24,3,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110...",5000000,10000000
1,20,Team Fortress Classic,Valve,Valve,6357,979,136,20,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271...",5000000,10000000
2,10,Counter-Strike,Valve,Valve,214651,5470,7720,232,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3...",10000000,20000000
3,70,Half-Life,Valve,Valve,85453,3070,391,168,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ...",2000000,5000000
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,617,71,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ...",5000000,10000000


#### Removing rows with missing values

In [56]:
steamspy_data.shape

(61164, 16)

In [57]:
#drop all raws with null values
steamspy_data.dropna(inplace=True)

In [58]:
steamspy_data.shape

(60792, 16)

In [59]:
steamspy_data.isnull().sum()

appid              0
name               0
developer          0
publisher          0
positive           0
negative           0
average_forever    0
median_forever     0
price              0
initialprice       0
discount           0
languages          0
genre              0
tags               0
owners_lowb        0
owners_uppb        0
dtype: int64

In [63]:
#drop rows where `name` is 'none'
steamspy_data = steamspy_data[steamspy_data['name'] != 'none']
steamspy_data.shape

(60790, 16)

#### Languages column

In [73]:
#checking unique languages in the `languages` column
df_languages = steamspy_data['languages'].str.split(', ').explode().unique()
print(df_languages)

['English' 'French' 'German' 'Italian' 'Spanish - Spain' 'Korean'
 'Russian' 'Simplified Chinese' 'Traditional Chinese' 'Dutch' 'Danish'
 'Finnish' 'Japanese' 'Norwegian' 'Polish' 'Portuguese - Portugal'
 'Swedish' 'Thai' 'Turkish' 'Bulgarian' 'Czech' 'Greek' 'Hungarian'
 'Portuguese - Brazil' 'Romanian' 'Spanish - Latin America' 'Ukrainian'
 'Vietnamese' '#lang_slovakian' 'Not supported'
 'Russian\r\nEnglish\r\nSpanish - Spain\r\nFrench\r\nJapanese\r\nCzech'
 '(all with full audio support)' 'Traditional Chinese (text only)'
 '#lang_german;' 'Spanish' '#lang_#lang_spanish*#lang_full_audio'
 '#lang_english (full audio)' 'Arabic' 'Persian'
 '#lang_#lang_russian*#lang_full_audio' 'Filipino' 'Indonesian' 'Basque'
 'Belarusian' 'Welsh' 'Catalan' 'Slovak' 'Latvian' 'Lithuanian'
 'Icelandic' 'Portuguese' 'Croatian' 'Georgian' 'Estonian' 'Slovenian'
 'Albanian' 'Hebrew' 'Hindi' 'Macedonian' 'Kazakh' 'Malay' 'Galician'
 'Serbian' 'Bangla' 'Punjabi (Gurmukhi)' 'Telugu' 'Luxembourgish'
 'Azerbaij

In [120]:
# Define language mapping and replace language names with proper ones
language_mapping = {
    '#lang_slovakian': 'Slovakian',
    '#lang_russian': 'Russian',
    '#lang_german;': 'German',
    '#lang_#lang_spanish*#lang_full_audio': 'Spanish',
    '#lang_english (full audio)': 'English',
    '#lang_russian*#lang_full_audio': 'Russian',
    'Spanish - Spain': 'Spanish',
    'Spanish - Latin America': 'Spanish',
    'Portuguese - Portugal': 'Portuguese',
    'Portuguese - Brazil': 'Portuguese',
}

# Replace language names within each cell
steamspy_data['languages'] = steamspy_data['languages'].str.replace('|'.join(language_mapping.keys()), lambda x: language_mapping[x.group()])


/var/folders/8c/2m03w30x1qd56mkg4zsspzfw0000gp/T/ipykernel_27511/892677419.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  steamspy_data['languages'] = steamspy_data['languages'].str.replace('|'.join(language_mapping.keys()), lambda x: language_mapping[x.group()])


In [123]:
#checking unique languages again in the `languages` column
df_languages = steamspy_data['languages'].str.split(', ').explode().unique()
print(df_languages)

['English' 'French' 'German' 'Italian' 'Spanish' 'Korean' 'Russian'
 'Simplified Chinese' 'Traditional Chinese' 'Dutch' 'Danish' 'Finnish'
 'Japanese' 'Norwegian' 'Polish' 'Portuguese' 'Swedish' 'Thai' 'Turkish'
 'Bulgarian' 'Czech' 'Greek' 'Hungarian' 'Romanian' 'Ukrainian'
 'Vietnamese' 'Slovakian' 'Not supported'
 'Russian\r\nEnglish\r\nSpanish\r\nFrench\r\nJapanese\r\nCzech'
 '(all with full audio support)' 'Traditional Chinese (text only)'
 '#lang_#lang_spanish*#lang_full_audio' '#lang_english (full audio)'
 'Arabic' 'Persian' '#lang_Russian*#lang_full_audio' 'Filipino'
 'Indonesian' 'Basque' 'Belarusian' 'Welsh' 'Catalan' 'Slovak' 'Latvian'
 'Lithuanian' 'Icelandic' 'Croatian' 'Georgian' 'Estonian' 'Slovenian'
 'Albanian' 'Hebrew' 'Hindi' 'Macedonian' 'Kazakh' 'Malay' 'Galician'
 'Serbian' 'Bangla' 'Punjabi (Gurmukhi)' 'Telugu' 'Luxembourgish'
 'Azerbaijani' 'Afrikaans' 'Dari' 'Mongolian' 'Urdu' 'Irish' 'Swahili'
 'Kannada' 'Tamil' 'Maori' 'Valencian' 'Bosnian' 'Scots' 'Gujarati'

In [124]:
# For some reason, some of the values weren't getting mapped, so they will be droped once each language gets it's own column

# Split languages into separate columns
language_dummies = steamspy_data['languages'].str.get_dummies(', ')

# Display the updated DataFrame
steamspy_data = pd.concat([steamspy_data, language_dummies], axis=1)
steamspy_data.shape

(60790, 98)

In [ ]:
#check column names to see which ones need to be dropped
column_names = list(steamspy_data.columns.values)
print(column_names)

In [ ]:
#drop columns with wrong names
steamspy_data = steamspy_data.drop(columns=['#lang_#lang_spanish*#lang_full_audio', 
                                            '#lang_Russian*#lang_full_audio', 
                                            '#lang_english (full audio)', 
                                            '(all with full audio support)', 
                                            'Not supported', 
                                            "Russian\r\nEnglish\r\nSpanish\r\nFrench\r\nJapanese\r\nCzech", 
                                            'Traditional Chinese (text only)']
                                            )

In [141]:
column_names = list(steamspy_data.columns.values)
print(column_names)

['appid', 'name', 'developer', 'publisher', 'positive', 'negative', 'average_forever', 'median_forever', 'price', 'initialprice', 'discount', 'languages', 'genre', 'tags', 'owners_lowb', 'owners_uppb', 'Afrikaans', 'Albanian', 'Arabic', 'Azerbaijani', 'Bangla', 'Basque', 'Belarusian', 'Bosnian', 'Bulgarian', 'Catalan', 'Croatian', 'Czech', 'Danish', 'Dari', 'Dutch', 'English', 'Estonian', 'Filipino', 'Finnish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malay', 'Maori', 'Marathi', 'Mongolian', 'Nepali', 'Norwegian', 'Persian', 'Polish', 'Portuguese', 'Punjabi (Gurmukhi)', 'Romanian', 'Russian', 'Scots', 'Serbian', 'Simplified Chinese', 'Slovak', 'Slovakian', 'Slovenian', 'Spanish', 'Swahili', 'Swedish', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Traditional Chinese', 'Turkish', 'Ukrain

In [142]:
# Specify the columns to sum
column_sums = steamspy_data.loc[:, 'Afrikaans':'Zulu'].sum()

# Calculate the sum for the specified columns
column_sums.sort_values(ascending=False)

English                60449
German                 15475
French                 14860
Simplified Chinese     14609
Spanish                14364
Russian                14127
Japanese               12004
Italian                10331
Portuguese              9584
Korean                  7616
Traditional Chinese     7219
Polish                  5955
Turkish                 4139
Dutch                   3410
Czech                   2578
Swedish                 2280
Ukrainian               2258
Hungarian               2114
Arabic                  2036
Thai                    1955
Danish                  1914
Norwegian               1821
Finnish                 1818
Romanian                1620
Greek                   1618
Bulgarian               1416
Vietnamese              1307
Indonesian               113
Slovak                    70
Catalan                   68
Hindi                     58
Hebrew                    40
Malay                     37
Belarusian                37
Serbian       

In [143]:
# Calculate the sum of values in the same row for the specified column range
languages_sum = steamspy_data.loc[:, 'Afrikaans':'Zulu'].sum(axis=1)

# Add the row sums as a new column in the DataFrame
steamspy_data['languages_sum'] = languages_sum

In [149]:
# Since we have a lot of languages, we are going to drop all languages that with sum less 1,000

# Filter the columns based on the sum condition
columns_to_drop = column_sums[column_sums < 1000].index

# Drop the columns from the DataFrame
steamspy_data = steamspy_data.drop(columns=columns_to_drop)

In [151]:
#drop original languages column
steamspy_data = steamspy_data.drop(columns=['languages'])

In [152]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,average_forever,median_forever,price,initialprice,discount,genre,tags,owners_lowb,owners_uppb,Arabic,Bulgarian,Czech,Danish,Dutch,English,Finnish,French,German,Greek,Hungarian,Italian,Japanese,Korean,Norwegian,Polish,Portuguese,Romanian,Russian,Simplified Chinese,Spanish,Swedish,Thai,Traditional Chinese,Turkish,Ukrainian,Vietnamese,languages_sum
0,60,Ricochet,Valve,Valve,4218,929,24,3,499.0,499.0,0.0,Action,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110...",5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,9
1,20,Team Fortress Classic,Valve,Valve,6357,979,136,20,499.0,499.0,0.0,Action,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271...",5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,9
2,10,Counter-Strike,Valve,Valve,214651,5470,7720,232,999.0,999.0,0.0,Action,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3...",10000000,20000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8
3,70,Half-Life,Valve,Valve,85453,3070,391,168,999.0,999.0,0.0,Action,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ...",2000000,5000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,617,71,999.0,999.0,0.0,Action,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ...",5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8


#### Tags column

In [166]:
#get tags as separate dataframe
tag = steamspy_data['tags']

tag.head()

0    {'Action': 588, 'FPS': 132, 'Multiplayer': 110...
1    {'Action': 757, 'FPS': 318, 'Multiplayer': 271...
2    {'Action': 5443, 'FPS': 4857, 'Multiplayer': 3...
3    {'FPS': 2306, 'Sci-fi': 1819, "1990's": 1816, ...
4    {'Action': 1369, 'FPS': 1027, 'Shooter': 759, ...
Name: tags, dtype: object

In [167]:
# using literal_eval we get values using keys from dictoninary in tags column
eval_row = literal_eval(tag[0])

print(eval_row['Action'])
print(eval_row['FPS'])

588
132


In [169]:
# Generate set of unique tag names

parsed_tags = tag.apply(lambda x: literal_eval(x))

cols = set(itertools.chain(*parsed_tags))

print('Number of unique tags:', len(cols))

Number of unique tags: 445


In [170]:
# Create separate tag dataframe

tag_data = steamspy_data[['appid', 'tags']].copy()

In [172]:
def parse_tags(x):
    x = literal_eval(x)
    
    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

tag_data['tags'] = tag_data['tags'].apply(parse_tags)
        
cols = set(itertools.chain(*tag_data['tags']))

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

    # check if column in row's dictionary of tags and return that value if it is, or 0 if it isn't
    tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

tag_data = tag_data.drop('tags', axis=1)

tag_data.to_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steamspy_tag.csv', index=False)


/var/folders/8c/2m03w30x1qd56mkg4zsspzfw0000gp/T/ipykernel_27511/980868003.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
/var/folders/8c/2m03w30x1qd56mkg4zsspzfw0000gp/T/ipykernel_27511/980868003.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
/var/folders/8c/2m03w30x1qd56mkg4zsspzfw0000gp/T/ipykernel_27511/980868003.py:20: PerformanceWarning: 

In [176]:
#adding top 3 tags to steamspy_data

def parse_tags(x):
    x = literal_eval(x)

    if isinstance(x, dict):
        return ';'.join(list(x.keys())[:3])
    else:
        return np.nan
    
steamspy_data['tags'] = steamspy_data['tags'].apply(parse_tags)

steamspy_data = steamspy_data[steamspy_data['tags'].notnull()]

In [177]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,average_forever,median_forever,price,initialprice,discount,genre,tags,owners_lowb,owners_uppb,Arabic,Bulgarian,Czech,Danish,Dutch,English,Finnish,French,German,Greek,Hungarian,Italian,Japanese,Korean,Norwegian,Polish,Portuguese,Romanian,Russian,Simplified Chinese,Spanish,Swedish,Thai,Traditional Chinese,Turkish,Ukrainian,Vietnamese,languages_sum
0,60,Ricochet,Valve,Valve,4218,929,24,3,499.0,499.0,0.0,Action,Action;FPS;Multiplayer,5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,9
1,20,Team Fortress Classic,Valve,Valve,6357,979,136,20,499.0,499.0,0.0,Action,Action;FPS;Multiplayer,5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,9
2,10,Counter-Strike,Valve,Valve,214651,5470,7720,232,999.0,999.0,0.0,Action,Action;FPS;Multiplayer,10000000,20000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8
3,70,Half-Life,Valve,Valve,85453,3070,391,168,999.0,999.0,0.0,Action,FPS;Sci-fi;1990's,2000000,5000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,617,71,999.0,999.0,0.0,Action,Action;FPS;Shooter,5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8


In [178]:
# inspect tag data
pd.read_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steamspy_tag.csv').head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,8_bit_music,atv,abstract,action,action_rpg,action_rts,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,ambient,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asymmetric_vr,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,baseball,based_on_a_novel,basketball,battle_royale,beat_em_up,beautiful,benchmark,bikes,birds,blood,board_game,boss_rush,bowling,boxing,building,bullet_hell,bullet_time,crpg,capitalism,card_battler,card_game,cartoon,cartoony,casual,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,coding,cold_war,collectathon,colony_sim,colorful,combat,combat_racing,comedy,comic_book,competitive,conspiracy,controller,conversation,cooking,cozy,crafting,creature_collector,cricket,crime,crowdfunded,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,deckbuilding,demons,design_&_illustration,destruction,detective,difficult,dinosaurs,diplomacy,documentary,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,early_access,economy,education,electronic,electronic_music,emotional,epic,episodic,escape_room,experience,experimental,exploration,fmv,fps,faith,family_friendly,fantasy,farming,farming_sim,fast_paced,feature_film,female_protagonist,fighting,first_person,fishing,flight,football,football_(american),football_(soccer),foreign,fox,free_to_play,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,heist,hentai,hero_shooter,hex_grid,hidden_object,historical,hobby_sim,hockey,horror,horses,hunting,idler,illuminati,immersive,immersive_sim,indie,instrumental_music,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jrpg,jet,job_simulator,jump_scare,kickstarter,lego,lgbtq+,lemmings,level_editor,life_sim,linear,local_co_op,local_multiplayer,logic,loot,looter_shooter,lore_rich,lovecraftian,mmorpg,moba,magic,mahjong,management,mars,martial_arts,massively_multiplayer,masterpiece,match_3,mature,mechs,medical_sim,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,mining,mod,moddable,modern,motocross,motorbike,mouse_only,movie,multiplayer,multiple_endings,music,music_based_procedural_generation,musou,mystery,mystery_dungeon,mythology,nsfw,narration,narrative,nature,naval,naval_combat,ninja,noir,nonlinear,nostalgia,nudity,offroad,old_school,on_rails_shooter,online_co_op,open_world,open_world_survival_craft,otome,outbreak_sim,parkour,parody_,party,party_game,party_based_rpg,perma_death,philosophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,point_&_click,political,political_sim,politics,pool,post_apocalyptic,precision_platformer,procedural_generation,programming,psychedelic,psychological,psychological_horror,puzzle,puzzle_platformer,pve,pvp,quick_time_events,rpg,rpgmaker,rts,racing,real_time_tactics,real_time,real_time_with_pause,realistic,reboot,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rock_music,rogue_like,rogue_lite,roguelike_deckbuilder,roguevania,romance,rome,rugby,runner,sailing,sandbox,satire,sci_fi,science,score_attack,sequel,sexual_content,shoot_em_up,shooter,shop_keeper,short,side_scroller,silent_protagonist,simulation,singleplayer,skateboarding,skating,skiing,sniper,snooker,snow,snowboarding,social_deduction,software,software_training,sokoban,solitaire,souls_like,soundtrack,space,space_sim,spaceships,spectacle_fighter,spelling,split_screen,sports,stealth,steam_machine,steampunk,story_rich,strategy,strategy_rpg,s

In [179]:
steamspy_data.isnull().sum()

appid                  0
name                   0
developer              0
publisher              0
positive               0
negative               0
average_forever        0
median_forever         0
price                  0
initialprice           0
discount               0
genre                  0
tags                   0
owners_lowb            0
owners_uppb            0
Arabic                 0
Bulgarian              0
Czech                  0
Danish                 0
Dutch                  0
English                0
Finnish                0
French                 0
German                 0
Greek                  0
Hungarian              0
Italian                0
Japanese               0
Korean                 0
Norwegian              0
Polish                 0
Portuguese             0
Romanian               0
Russian                0
Simplified Chinese     0
Spanish                0
Swedish                0
Thai                   0
Traditional Chinese    0
Turkish                0


In [180]:
# Export clean SteamSpy data to CSV

steamspy_data.to_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steamspy_clean.csv', index=False)

## Steam Data

In [181]:
# read in downloaded data
raw_steam_data = pd.read_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steam_data_VA_raw.csv')

/var/folders/8c/2m03w30x1qd56mkg4zsspzfw0000gp/T/ipykernel_27511/4161852919.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_steam_data = pd.read_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steam_data_VA_raw.csv')


In [182]:
raw_steam_data.dtypes

type                        object
name                        object
steam_appid                  int64
required_age                object
is_free                     object
controller_support          object
dlc                         object
detailed_description        object
about_the_game              object
short_description           object
fullgame                   float64
supported_languages         object
header_image                object
website                     object
pc_requirements             object
mac_requirements            object
linux_requirements          object
legal_notice                object
drm_notice                  object
ext_user_account_notice     object
developers                  object
publishers                  object
demos                       object
price_overview              object
packages                    object
package_groups              object
platforms                   object
metacritic                  object
reviews             

In [185]:
# print out number of rows and columns
print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])

Rows: 66803
Columns: 39


In [184]:
raw_steam_data.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,3D Ultra™ Minigolf Adventures,10140,0.0,False,NaN,NaN,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],NaN,NaN,NaN,['Wanako Games'],['Activision'],NaN,"{'currency': 'EUR', 'initial': 999, 'final': 9...",[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '18', 'description': 'Sports'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,NaN,"{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,A Stroke of Fate: Operation Valkyrie,10240,0.0,False,NaN,NaN,<p>Operation Valkyrie is a famous plan of the ...,<p>Operation Valkyrie is a famous plan of the ...,Operation Valkyrie is a famous plan of the Ger...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],© 2011 Akella. All rights reserved.,NaN,NaN,['SPLine'],['Akella'],NaN,"{'currency': 'EUR', 'initial': 699, 'final': 6...",[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,NaN,"{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,game,Dracula: Origin,11050,0.0,False,NaN,NaN,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],© 2008 Frogwares Ireland ltd. All rights reser...,NaN,NaN,['Frogwares'],['Frogwares'],"[{'appid': 11090, 'description': ''}]","{'currency': 'EUR', 'initial': 999, 'final': 9...",[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","{'score': 70, 'url': 'https://www.metacritic.c...",“Dracula: Origin is elegant piece of work visu...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 900601, 'name': 'Dracula: Origin Trail...",{'total': 166},NaN,"{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Gumboy Tournament,11230,0.0,False,NaN,NaN,"Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...",NaN,"English, German, Polish, Czech",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],"2006-2008 © CINEMAX, s.r.o.",NaN,NaN,"['CINEMAX, s.r.o.']","['CINEMAX, s.r.o.']","[{'appid': 11270, 'description': ''}]","{'currency': 'EUR

In [186]:
#checking Null values in all columns
raw_steam_data.isnull().sum()

type                         148
name                           0
steam_appid                    0
required_age                 148
is_free                      148
controller_support         52381
dlc                        57461
detailed_description         216
about_the_game               218
short_description            210
fullgame                   66803
supported_languages          204
header_image                 148
website                    30684
pc_requirements              148
mac_requirements             148
linux_requirements           148
legal_notice               47301
drm_notice                 66525
ext_user_account_notice    65679
developers                   308
publishers                   148
demos                      60794
price_overview             19453
packages                   18840
package_groups               148
platforms                    148
metacritic                 63077
reviews                    57931
categories                  1104
genres    

In [191]:
# Checking columns with more than 50% of the values missing
threshold = int(raw_steam_data.shape[0] * 0.5)
null_counts = raw_steam_data.isnull().sum()
drop_columns = raw_steam_data.columns[null_counts > threshold]

print('Droping columns: {}'.format(list(drop_columns)))

Droping columns: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations', 'achievements']


In [193]:
# Drop columns
steam_data = raw_steam_data.drop(columns=drop_columns)
print('Number of columns: {}'.format(steam_data.shape[1]))

Number of columns: 28


In [199]:
# Type column
print('Number of rows with values missing in type column: {}'.format(steam_data['type'].isnull().sum()))
print('Number of different values: {}'.format(steam_data['type'].value_counts(dropna=False)))

# Check rows with null values
steam_data[steam_data['type'].isnull()].head(3)

Number of rows with values missing in type column: 148
Number of different values: game           66653
NaN              148
advertising        1
hardware           1
Name: type, dtype: int64


,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors
535,NaN,(Chinese PaladinSword and Fairy 6),681810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,NaN,(Chinese PaladinSword and Fairy 4),681820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,NaN,(Chinese PaladinSword and Fairy 5),681830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
#since we are interested only in games, we will drop rows with hardware and advertising, and rows with null values (there is no data)
steam_data = steam_data[steam_data['type'] == 'game']

print('Number of rows with values missing in type column: {}'.format(steam_data['type'].isnull().sum()))
print('Number of different values: {}'.format(steam_data['type'].value_counts(dropna=False)))

Number of rows with values missing in type column: 0
Number of different values: game    66653
Name: type, dtype: int64


In [201]:
# since now there's only one value in type column, we will drop it
steam_data = steam_data.drop(columns='type')
steam_data.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0.0,False,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['Wanako Games'],['Activision'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '18', 'description': 'Sports'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0.0,False,<p>Operation Valkyrie is a famous plan of the ...,<p>Operation Valkyrie is a famous plan of the ...,Operation Valkyrie is a famous plan of the Ger...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],['SPLine'],['Akella'],"{'currency': 'EUR', 'initial': 699, 'final': 6...",[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0.0,False,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],['Frogwares'],['Frogwares'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 900601, 'name': 'Dracula: Origin Trail...","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0.0,False,"Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","English, German, Polish, Czech",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],"['CINEMAX, s.r.o.']","['CINEMAX, s.r.o.']","{'currency': 'EUR', 'initial': 199, 'final': 1...",[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '19 May, 2008'}",{'url': 'http://feedback.cinemax.cz/?email=Gum...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0.0,False,Big Score is based around a classic story of p...,Big Score is based around a

In [206]:
# name column 
steam_data[(steam_data['name'].isnull()) | (steam_data['name'] == 'none')]

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors
65991,none,339860,0.0,False,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,[''],NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,"{'coming_soon': False, 'date': '27 Feb, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
66127,none,385020,0.0,False,- discontinued - (please remove),- discontinued - (please remove),- discontinued - (please remove),"English, French, Italian, German, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,['none'],[''],NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...",NaN,NaN,"{'coming_soon': False, 'date': '4 Nov, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
66173,none,398970,0.0,False,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,['none'],['none'],NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,"{'coming_soon': False, 'date': '5 Nov, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"


In [211]:
#drop rows where `name` is 'none' or NaN
steam_data = steam_data[steam_data['name'].notnull()]
steam_data = steam_data[steam_data['name'] != 'none']

In [213]:
steam_data.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0.0,False,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['Wanako Games'],['Activision'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '18', 'description': 'Sports'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0.0,False,<p>Operation Valkyrie is a famous plan of the ...,<p>Operation Valkyrie is a famous plan of the ...,Operation Valkyrie is a famous plan of the Ger...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],['SPLine'],['Akella'],"{'currency': 'EUR', 'initial': 699, 'final': 6...",[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0.0,False,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],['Frogwares'],['Frogwares'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 900601, 'name': 'Dracula: Origin Trail...","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0.0,False,"Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","English, German, Polish, Czech",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],"['CINEMAX, s.r.o.']","['CINEMAX, s.r.o.']","{'currency': 'EUR', 'initial': 199, 'final': 1...",[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '19 May, 2008'}",{'url': 'http://feedback.cinemax.cz/?email=Gum...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0.0,False,Big Score is based around a classic story of p...,Big Score is based around a

#### Age

In [215]:
steam_data['required_age'].value_counts(dropna=False)

0                                                                             41454
0.0                                                                           17185
0.0                                                                            6968
18                                                                              242
18.0                                                                            195
16                                                                              105
16.0                                                                            103
18.0                                                                             84
16.0                                                                             50
12                                                                               43
12.0                                                                             36
17.0                                                                        

In [229]:
#since there are multiple values that are 
steam_data['required_age'].replace({'18+': 18,
                                    'javascript:ToggleCheckbox(\'checkbox_app_game_ratings_pegi_use_age_gate_\');': 0,
                                    ' 1 8 ': 18,
                                    '99999': 0,
                                    '0.0': 0,
                                    '17.0': 17},
                                   inplace=True)

In [233]:
#change column to integer
steam_data['required_age'] = steam_data['required_age'].astype(float)
steam_data['required_age'] = steam_data['required_age'].astype(int)

0     65609
18      524
16      258
12       96
17       51
15       31
13       24
7        18
3        17
10        7
14        4
11        3
20        2
6         2
1         1
4         1
5         1
19        1
Name: required_age, dtype: int64

In [237]:
# grouping age by ESRB rating system: 10, 13, 17, 18
cut_points = [-1, 0, 10, 13, 17, 18]
label_values = [0, 10, 13, 17, 18]
    
steam_data['required_age'] = pd.cut(steam_data['required_age'], bins=cut_points, labels=label_values)

In [238]:
steam_data['required_age'].value_counts()

0     65609
18      524
17      344
13      123
10       47
Name: required_age, dtype: int64

In [239]:
steam_data.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0,False,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,Whether you're a seasoned player or just a beg...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['Wanako Games'],['Activision'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '18', 'description': 'Sports'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,<p>Operation Valkyrie is a famous plan of the ...,<p>Operation Valkyrie is a famous plan of the ...,Operation Valkyrie is a famous plan of the Ger...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],['SPLine'],['Akella'],"{'currency': 'EUR', 'initial': 699, 'final': 6...",[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0,False,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,Dracula: Origin reveals the dark origins of th...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],['Frogwares'],['Frogwares'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 900601, 'name': 'Dracula: Origin Trail...","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0,False,"Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","Test your reflexes, judgment and tactics while...","English, German, Polish, Czech",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],"['CINEMAX, s.r.o.']","['CINEMAX, s.r.o.']","{'currency': 'EUR', 'initial': 199, 'final': 1...",[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '19 May, 2008'}",{'url': 'http://feedback.cinemax.cz/?email=Gum...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0,False,Big Score is based around a classic story of p...,Big Score is based around a classic s

In [241]:
# changing 'is_free' column in boolean
steam_data['is_free'] = steam_data['is_free'].astype(bool)

#### Exporting desctiption columns

In [243]:
steam_data[['detailed_description', 'about_the_game', 'short_description']].isnull().sum()

detailed_description    66
about_the_game          68
short_description       60
dtype: int64

In [244]:
# create dataframe of description columns
description_data = steam_data[['steam_appid', 'detailed_description', 'about_the_game', 'short_description']]

description_data.to_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steam_description_data.csv', index=False)

# drop description columns from main dataframe
steam_data = steam_data.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

steam_data.head()

,name,steam_appid,required_age,is_free,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0,False,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],['Wanako Games'],['Activision'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '18', 'description': 'Sports'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],['SPLine'],['Akella'],"{'currency': 'EUR', 'initial': 699, 'final': 6...",[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0,False,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],['Frogwares'],['Frogwares'],"{'currency': 'EUR', 'initial': 999, 'final': 9...",[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 900601, 'name': 'Dracula: Origin Trail...","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0,False,"English, German, Polish, Czech",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],"['CINEMAX, s.r.o.']","['CINEMAX, s.r.o.']","{'currency': 'EUR', 'initial': 199, 'final': 1...",[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '19 May, 2008'}",{'url': 'http://feedback.cinemax.cz/?email=Gum...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0,False,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.slamitpinball.com/home.htm,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...",[],[],['Technetium Games'],['Kalypso Media Digital'],"{'currency': 'EUR', 'initial': 499, 'final': 4...",[4054],"[{'name': 'default', 'title': 'Buy SlamIt Pinb...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Feb, 2009'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"


### Dropping columns that we already have in SteamSpy dataset

In [245]:
# drop columns since we already have columns in SteamSpy datase and data is more recent 'language', 'genre', 'developer', 'publisher', 'price'
steam_data = steam_data.drop(['supported_languages', 'developers', 'publishers', 'price_overview', 'genres'], axis=1) # provided by SteamSpy dataset
steam_data.head()

,name,steam_appid,required_age,is_free,header_image,website,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,platforms,categories,screenshots,movies,release_date,support_info,background,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0,False,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,https://cdn.akamai.steamstatic.com/steam/apps/...,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0,False,https://cdn.akamai.steamstatic.com/steam/apps/...,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 900601, 'name': 'Dracula: Origin Trail...","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0,False,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '19 May, 2008'}",{'url': 'http://feedback.cinemax.cz/?email=Gum...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0,False,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.slamitpinball.com/home.htm,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...",[],[],[4054],"[{'name': 'default', 'title': 'Buy SlamIt Pinb...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': '23 Feb, 2009'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"


### Media columns

In [246]:
steam_data[['header_image', 'screenshots', 'movies', 'background']].isnull().sum()

header_image       0
screenshots      119
movies          4978
background       105
dtype: int64

In [247]:
# create dataframe of description columns
media_data = steam_data[['steam_appid', 'header_image', 'screenshots', 'movies', 'background']]

# exporting media columns in CSV
media_data.to_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steam_media_data.csv', index=False)

# drop description columns from main dataframe
steam_data = steam_data.drop(['header_image', 'screenshots', 'movies', 'background'], axis=1)

steam_data.head()

,name,steam_appid,required_age,is_free,website,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,platforms,categories,release_date,support_info,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0,False,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Jul, 2009'}","{'url': 'http://support.activision.com/', 'ema...","{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,http://astrokeoffate.akella.com/,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '28 Oct, 2011'}","{'url': '', 'email': 'support@akella.com'}","{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0,False,http://frogwares.com/,{'minimum': '<strong>Minimum:</strong> ...,[],[],[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'url': 'http://frogwares.com/support/', 'emai...","{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0,False,http://www.gumboytournament.com/,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': False, 'date': '19 May, 2008'}",{'url': 'http://feedback.cinemax.cz/?email=Gum...,"{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0,False,http://www.slamitpinball.com/home.htm,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...",[],[],[4054],"[{'name': 'default', 'title': 'Buy SlamIt Pinb...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Feb, 2009'}","{'url': '', 'email': ''}","{'ids': [], 'notes': None}"


### Website columns

In [248]:
steam_data[['website', 'support_info']].isnull().sum()

website         30533
support_info        0
dtype: int64

In [250]:
# create dataframe of description columns
website_data = steam_data[['steam_appid', 'website', 'support_info']]

# exporting media columns in CSV
website_data.to_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steam_website_data.csv', index=False)

# drop description columns from main dataframe
steam_data = steam_data.drop(['website', 'support_info'], axis=1)

steam_data.head()

,name,steam_appid,required_age,is_free,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,platforms,categories,release_date,content_descriptors
0,3D Ultra™ Minigolf Adventures,10140,0,False,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Jul, 2009'}","{'ids': [], 'notes': None}"
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,{'minimum': '<ul>\n\t\t\t\t\t\t\t<li><strong>O...,[],[],[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '28 Oct, 2011'}","{'ids': [], 'notes': None}"
2,Dracula: Origin,11050,0,False,{'minimum': '<strong>Minimum:</strong> ...,[],[],[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '7 Aug, 2008'}","{'ids': [], 'notes': None}"
3,Gumboy Tournament,11230,0,False,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': False, 'date': '19 May, 2008'}","{'ids': [], 'notes': None}"
4,SlamIt Pinball Big Score,12430,0,False,"{'minimum': '<ul class=""bb_ul""><li><strong>Sup...",[],[],[4054],"[{'name': 'default', 'title': 'Buy SlamIt Pinb...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Feb, 2009'}","{'ids': [], 'notes': None}"


### content_descriptors column

In [252]:
steam_data['content_descriptors'].value_counts().sum()

66650

In [254]:
# all values are same we are dropping column
steam_data = steam_data.drop('content_descriptors', axis=1)

### System Requirements

In [255]:
steam_data[['pc_requirements', 'mac_requirements', 'linux_requirements']].isnull().sum()

pc_requirements       0
mac_requirements      0
linux_requirements    0
dtype: int64

In [256]:
#since we don't need this data for analysis, we will export columns

# create dataframe of description columns
system_req_data = steam_data[['steam_appid', 'pc_requirements', 'mac_requirements', 'linux_requirements']]

# exporting media columns in CSV
system_req_data.to_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steam_system_req_data.csv', index=False)

# drop description columns from main dataframe
steam_data = steam_data.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)

steam_data.head()

,name,steam_appid,required_age,is_free,packages,package_groups,platforms,categories,release_date
0,3D Ultra™ Minigolf Adventures,10140,0,False,[1873],"[{'name': 'default', 'title': 'Buy 3D Ultra™ M...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Jul, 2009'}"
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,[11852],"[{'name': 'default', 'title': 'Buy A Stroke of...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '28 Oct, 2011'}"
2,Dracula: Origin,11050,0,False,[905],"[{'name': 'default', 'title': 'Buy Dracula: Or...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '7 Aug, 2008'}"
3,Gumboy Tournament,11230,0,False,[768],"[{'name': 'default', 'title': 'Buy Gumboy Tour...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': False, 'date': '19 May, 2008'}"
4,SlamIt Pinball Big Score,12430,0,False,[4054],"[{'name': 'default', 'title': 'Buy SlamIt Pinb...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Feb, 2009'}"


### Packages columns

In [258]:
# we won't use those columns in our analysis so we will drop them
steam_data = steam_data.drop(['packages', 'package_groups'], axis=1)

In [267]:
# Convert string values to dictionaries
steam_data['platforms'] = steam_data['platforms'].apply(literal_eval)

# Create separate columns for windows, mac, and linux
steam_data['windows'] = pd.Series(steam_data['platforms']).apply(lambda x: x.get('windows', False))
steam_data['mac'] = pd.Series(steam_data['platforms']).apply(lambda x: x.get('mac', False))
steam_data['linux'] = pd.Series(steam_data['platforms']).apply(lambda x: x.get('linux', False))

#drop original platforms column
steam_data = steam_data.drop(['platforms'], axis=1)

steam_data.head()

,name,steam_appid,required_age,is_free,categories,release_date,windows,mac,linux
0,3D Ultra™ Minigolf Adventures,10140,0,False,"[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Jul, 2009'}",True,False,False
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,"[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '28 Oct, 2011'}",True,False,False
2,Dracula: Origin,11050,0,False,"[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '7 Aug, 2008'}",True,False,False
3,Gumboy Tournament,11230,0,False,"[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': False, 'date': '19 May, 2008'}",True,False,False
4,SlamIt Pinball Big Score,12430,0,False,"[{'id': 2, 'description': 'Single-player'}]","{'coming_soon': False, 'date': '23 Feb, 2009'}",True,False,False


### Categories column

In [271]:
print(steam_data['categories'].isnull().sum())
print(steam_data['categories'][0])

955
[{'id': 2, 'description': 'Single-player'}]


In [273]:
steam_data[steam_data['categories'].isnull()].sample(5, random_state=1)

,name,steam_appid,required_age,is_free,categories,release_date,windows,mac,linux
64591,InductVR (beta),907180,0,True,NaN,"{'coming_soon': False, 'date': '2 Jun, 2019'}",True,False,False
66465,VEGAS Pro 14 Edit Steam Edition,528200,0,False,NaN,"{'coming_soon': False, 'date': '30 Sep, 2016'}",True,False,False
65142,Masterpiece Studio,1165780,0,False,NaN,"{'coming_soon': False, 'date': '16 Dec, 2019'}",True,False,False
65061,Transparent Taskbar,1113620,0,False,NaN,"{'coming_soon': False, 'date': '13 Jul, 2019'}",True,False,False
65517,RPG Sounds,1480140,0,True,NaN,"{'coming_soon': False, 'date': '27 Jan, 2021'}",True,True,False


In [275]:
# remove rows with null values
steam_data = steam_data[steam_data['categories'].notnull()]
print(steam_data['categories'].isnull().sum())

0


In [276]:
# Convert the 'categories' column from string to a list of dictionaries
steam_data['categories'] = steam_data['categories'].apply(literal_eval)

# Extract 'description' values and join them with semicolons
steam_data['categories'] = steam_data['categories'].apply(lambda x: ';'.join([category['description'] for category in x]))

steam_data.head()


,name,steam_appid,required_age,is_free,categories,release_date,windows,mac,linux
0,3D Ultra™ Minigolf Adventures,10140,0,False,Single-player,"{'coming_soon': False, 'date': '23 Jul, 2009'}",True,False,False
1,A Stroke of Fate: Operation Valkyrie,10240,0,False,Single-player,"{'coming_soon': False, 'date': '28 Oct, 2011'}",True,False,False
2,Dracula: Origin,11050,0,False,Single-player,"{'coming_soon': False, 'date': '7 Aug, 2008'}",True,False,False
3,Gumboy Tournament,11230,0,False,Single-player;Multi-player,"{'coming_soon': False, 'date': '19 May, 2008'}",True,False,False
4,SlamIt Pinball Big Score,12430,0,False,Single-player,"{'coming_soon': False, 'date': '23 Feb, 2009'}",True,False,False


## Merge data

In [ ]:
steam_data = pd.read_csv('../data/exports/steam_data_clean.csv')
    
merged = steam_data.merge(steamspy_data, left_on='steam_appid', right_on='appid', suffixes=('', '_steamspy'))
merged.head()

In [ ]:
# remove overlapping columns
steam_clean = merged.drop(['name_steamspy', 'languages', 'steam_appid'], axis=1)

# reindex to reorder columns
steam_clean = steam_clean[[
    'appid',
    'name',
    'release_date',
    'english',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'tags',
    'achievements',
    'positive',
    'negative',
    'average_forever',
    'median_forever',
    'owners',
    'price'
]]

steam_clean = steam_clean.rename({
    'tags': 'steamspy_tags',
    'positive': 'positive_ratings',
    'negative': 'negative_ratings',
    'average_forever': 'average_playtime',
    'median_forever': 'median_playtime'
}, axis=1)

steam_clean.head()